In [2]:
import string
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

UPPER_BOUND=2
LOWER_BOUND=-2
LEFT_BOUND=-0.5
RIGHT_BOUND=1.5
BACK_BOUND=0
FRONT_BOUND=0.1

class Face(Enum):
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4
    FRONT = 5
    BACK = 6

class Airfoil:
    """ NACA Airfoil (MPXX, IT) """
    def __init__(self, mpxx: string, points: int):

        self.__mpxx = mpxx
        self.__points = points - 1

        self.__MaxCamber = float(self.__mpxx[0])/100
        self.__MaxCamberPosition = float(self.__mpxx[1])/10
        self.__Thickness = float(self.__mpxx[2:])/100

        self.GetAirFoilParams()
        self.CalculatePoints()

    def GetAirFoilParams(self):
        print("__ NACA AIRFOIL {0}, MaxCamber(%) {1}, MaxCamberPosition(%) {2} Thickness(%) :{3} __ Points {4}".format(
                                                                                                self.__mpxx,
                                                                                                self.__MaxCamber,
                                                                                                self.__MaxCamberPosition,
                                                                                                self.__Thickness,
                                                                                                self.__points+1) )
    
    def CalculatePoints(self):
        DelPoint = 0

        it = self.__points
        m = self.__MaxCamber
        p = self.__MaxCamberPosition
        t = self.__Thickness

        x = 0 + np.arange(0, it + (1/it)) * (1/it)
        
        yc = np.zeros(x.shape)
        firstTime = True
        
        for i in range(0, len(yc), 1):        
            if(x[i] < p):
                yc[i] = m / pow(p,2) * (2*p*x[i] - pow(x[i],2))
                firstTime = True
            else:
                if(firstTime):
                    DelPoint = i
                    firstTime = False
                yc[i] = m / pow((1-p),2) * (1 - 2*p + 2*p*x[i] - pow(x[i],2))
        
        theta = np.zeros(x.shape)
        dydx = np.zeros(x.shape)
        
        for i in range(0, len(yc), 1):        
            if(x[i] < p):
                dydx[i] = 2 * m / pow(p,2) * (p-x[i])
            else:
                dydx[i] = 2 * m / pow((1-p),2) * (p-x[i])

            theta[i] = np.arctan(dydx[i])

        yt = t/0.2 * (
                0.2969 * np.sqrt(x) 
            - 0.1260 * x 
            - 0.3516 * pow(x,2) 
            + 0.2843 * pow(x,3)
            - 0.1015 * pow(x,4))
            #- 0.1036 * pow(x,4))
        
        xu = np.zeros(x.shape)
        xl = np.zeros(x.shape)

        yu = np.zeros(x.shape)
        yl = np.zeros(x.shape)
        
        for i in range(0, len(yc), 1):     
            xu[i] = x[i] - yt[i] * np.sin(theta[i])
            xl[i] = x[i] + yt[i] * np.sin(theta[i])

            yu[i] = yc[i] + yt[i] * np.cos(theta[i])
            yl[i] = yc[i] - yt[i] * np.cos(theta[i])

        DPIup = 0
        DPIlw = 0
        
        for i in range(len(xu)-1, 0, -1):
            if(xu[i] < xu[DelPoint]):
                DPIup = i
                break
        
        for i in range(len(xl)-1, 0, -1):
            if(xl[i] < xl[DelPoint]):
                DPIlw = i
                break

        self.__XUSplittingPoint = xu[DPIup]
        self.__YUSplittingPoint = yu[DPIup]
    
        self.__XLSplittingPoint = xl[DPIlw]
        self.__YLSplittingPoint = yl[DPIlw]

        self.__DelPointUp = DPIup
        self.__DelPointLow = DPIlw

        self.__XUPoints = xu
        self.__YUPoints = yu

        self.__XLPoints = xl
        self.__YLPoints = yl

    def GetAllPoints(self):
        return self.__XUPoints, self.__YUPoints, self.__XLPoints, self.__YLPoints
    
    def GetSplittingPoints(self):
        return self.__XUSplittingPoint, self.__YUSplittingPoint, self.__XLSplittingPoint, self.__YLSplittingPoint
    
    def GetXU(self):
        return self.__XUPoints

    def GetXL(self):
        return self.__XLPoints

    def GetYU(self):
        return self.__YUPoints
    
    def GetYL(self):
        return self.__YLPoints
    
    def GetDelPoints(self):
        return self.__DelPointUp, self.__DelPointLow

    __XUPoints = []
    __YUPoints = []

    __XLPoints = []
    __YLPoints = []

    __XUSplittingPoint = 0
    __YUSplittingPoint = 0

    __XLSplittingPoint = 0
    __YLSplittingPoint = 0

    __DelPointUp = 0
    __DelPointLow = 0

class AirfoilMesh:
    def __init__(self, airfoil: Airfoil, blocksAmount : int ):
        self.__blocksAmount = blocksAmount
        self.__airfoil = airfoil

        self.__xu = self.__airfoil.GetXU()
        self.__xl = self.__airfoil.GetXL()
        self.__yu = self.__airfoil.GetYU()
        self.__yl = self.__airfoil.GetYL()

        self.CalculateSplines()
        self.CalculateBlocks()
        self.CalculateVertices()
        self.CalculateBoundaries()
        self.CalculateArcs()

    def CalculateVertices(self):

        XuSplits = []
        YuSplits = []
        XlSplits = []
        YlSplits = []

        splits = self.__blocksAmount // 2
        step = int(((1)*(len(self.__xu)))/(splits))

        for i in range(0, len(self.__xu), step):
            XuSplits.append(self.__xu[i])
            YuSplits.append(self.__yu[i])
            XlSplits.append(self.__xl[i])
            YlSplits.append(self.__yl[i])


        XuSplits.append(self.__xu[len(self.__xu)-1])
        YuSplits.append(self.__yu[len(self.__xu)-1])
        XlSplits.append(self.__xl[len(self.__xu)-1])
        YlSplits.append(self.__yl[len(self.__xu)-1])

        __DPIup, __DPIlow = self.__airfoil.GetDelPoints()

        XuSplits[1] = self.__xu[__DPIup]
        YuSplits[1] = self.__yu[__DPIup]

        XlSplits[1] = self.__xl[__DPIlow]
        YlSplits[1] = self.__yl[__DPIlow]

        for i in range(0, len(XuSplits), 1):
            self.__Vertices.append([XuSplits[i], YuSplits[i], BACK_BOUND])
            self.__VerticesAmount +=1
        
        for i in range(0, len(XuSplits), 1):
            self.__Vertices.append([XuSplits[i], UPPER_BOUND, BACK_BOUND])
            self.__VerticesAmount +=1

        for i in range(0, len(XuSplits), 1):
            self.__Vertices.append([XuSplits[i], YuSplits[i], FRONT_BOUND])
            self.__VerticesAmount +=1

        for i in range(0, len(XuSplits), 1):
            self.__Vertices.append([XuSplits[i], UPPER_BOUND, FRONT_BOUND])
            self.__VerticesAmount +=1
        
        ##################################################################

        for i in range(1, len(XlSplits)-1, 1):
            self.__Vertices.append([XlSplits[i], YlSplits[i], BACK_BOUND])
            self.__VerticesAmount +=1

        for i in range(1, len(XlSplits), 1):
            self.__Vertices.append([XlSplits[i], LOWER_BOUND, BACK_BOUND])
            self.__VerticesAmount +=1

        for i in range(1, len(XlSplits)-1, 1):
            self.__Vertices.append([XlSplits[i], YlSplits[i], FRONT_BOUND])
            self.__VerticesAmount +=1
        
        for i in range(1, len(XlSplits), 1):
            self.__Vertices.append([XlSplits[i], LOWER_BOUND, FRONT_BOUND])
            self.__VerticesAmount +=1


        

        self.__Vertices[splits+1][0] = -(UPPER_BOUND-(XuSplits[1] + XlSplits[1])/2)
        self.__Vertices[splits+1][1] = 0

        self.__Vertices[3 * (splits+1)][0] = -(UPPER_BOUND-(XuSplits[1] + XlSplits[1])/2)
        self.__Vertices[3 * (splits+1)][1] = 0
          
    def CalculateSplines(self):

        splits = self.__blocksAmount // 2
        step = int(((1)*(len(self.__xu)))/(splits))
        
        USplitsIndexes = []
        LSplitsIndexes = []
        
        DelPointUp, DelPointLow = self.__airfoil.GetDelPoints()
        
        for i in range(0, len(self.__xu) , step):
            USplitsIndexes.append(i)
            LSplitsIndexes.append(i)

        USplitsIndexes[1] = DelPointUp
        LSplitsIndexes[1] = DelPointLow

        USplitsIndexes.append(len(self.__xu))
        LSplitsIndexes.append(len(self.__xu))

        for i in range(0, splits, 1):
            self.__SplinePoints.append([i, i+1])
            self.__Splines.append(list(zip(
                self.__xu[USplitsIndexes[i] : USplitsIndexes[i+1] + 1],
                self.__yu[USplitsIndexes[i] : USplitsIndexes[i+1] + 1],
                [BACK_BOUND] * (USplitsIndexes[i+1]+1 - USplitsIndexes[i])
            )))
            self.__SplinesAmount += 1
        
        for i in range(0, splits, 1):
            self.__SplinePoints.append([i + (splits+1)*2, i + (splits+1)*2 + 1])
            self.__Splines.append(list(zip(
                self.__xu[USplitsIndexes[i] : USplitsIndexes[i+1] + 1],
                self.__yu[USplitsIndexes[i] : USplitsIndexes[i+1] + 1],
                [FRONT_BOUND] * (USplitsIndexes[i+1]+1 - USplitsIndexes[i])
            )))
            self.__SplinesAmount += 1

        
        for i in range(0, splits, 1):
            self.__SplinePoints.append([i, i+1])
            self.__Splines.append(list(zip(
                self.__xl[LSplitsIndexes[i] : LSplitsIndexes[i+1] + 1],
                self.__yl[LSplitsIndexes[i] : LSplitsIndexes[i+1] + 1],
                [BACK_BOUND] * (LSplitsIndexes[i+1]+1 - LSplitsIndexes[i])
            )))
            self.__SplinesAmount += 1
        
        for i in range(0, splits, 1):
            self.__SplinePoints.append([i + (splits+1)*2, i + (splits+1)*2 + 1])
            self.__Splines.append(list(zip(
                self.__xl[LSplitsIndexes[i] : LSplitsIndexes[i+1] + 1],
                self.__yl[LSplitsIndexes[i] : LSplitsIndexes[i+1] + 1],
                [FRONT_BOUND] * (LSplitsIndexes[i+1]+1 - LSplitsIndexes[i])
            )))
            self.__SplinesAmount += 1

        self.__SplinePoints[splits * 2] = [0, 12]
        self.__SplinePoints[splits * 2 + 1] = [12, 2]
        
        self.__SplinePoints[splits * 2 + 2] = [6, 15]
        self.__SplinePoints[splits * 2 + 3 ] = [15, 8]

    def CalculateBlocks(self):
        
        splits = self.__blocksAmount // 2
        UpperOrder = [[0 for i in range(8)] for j in range(splits)]
        LowerOrder = [[0 for i in range(8)] for j in range(splits)]

        u_preset = [0, 1,
                splits+2,
                splits+1,
                (splits+1)*2,
                (splits+1)*2 + 1,
                (splits+1)*3 + 1 ,
                (splits+1)*3]
        
        l_preset = [
              (splits + 1)*4 + (splits-1),
              (splits + 1)*4 + (splits-1),
              (splits + 1)*4,
              (splits + 1)*4 - 1,
              (splits + 1)*5 + (splits-1)*2,
              (splits + 1)*5 + (splits-1)*2,
              (splits + 1)*6 - 2,
              (splits + 1)*6 - 3
              ]

        
        for i in range(0, splits, 1):
            for j in range(0, 8, 1):
                UpperOrder[i][j] = u_preset[j] + i
                LowerOrder[i][j] = l_preset[j] + i
            

        LowerOrder[0][0] = 3
        LowerOrder[0][3] = 0

        LowerOrder[0][4] = 9
        LowerOrder[0][7] = 6

        LowerOrder[0][6] -=1
        LowerOrder[0][5] -=1
        

        ##################

        LowerOrder[len(LowerOrder)-1][0] -=1
        LowerOrder[len(LowerOrder)-1][4] -=2
        LowerOrder[len(LowerOrder)-1][5] -=1
        LowerOrder[len(LowerOrder)-1][7] -=1
        

        LowerOrder[len(LowerOrder)-1][2] = 2
        LowerOrder[len(LowerOrder)-1][6] = 8

        for i in range(0, splits, 1):
            self.__Blocks.append(UpperOrder[i])
            self.__BlocksAmount += 1
        
        for i in range(0, splits, 1):
            self.__Blocks.append(LowerOrder[i])
            self.__BlocksAmount += 1

        print(LowerOrder[1])
        
    def CalculateArcs(self):
        dUpPoint, dLowPoint = self.__airfoil.GetDelPoints()
        midPoint = (self.__xu[dUpPoint] + self.__xl[dLowPoint])/2

        arcUpBack = dict(boundPoints = [3,4], arcPoint = [-((UPPER_BOUND * np.sqrt(2)/2) - midPoint), ( UPPER_BOUND * np.sqrt(2)/2), BACK_BOUND])
        arcUpFront = dict(boundPoints = [9,10], arcPoint = [-((UPPER_BOUND * np.sqrt(2)/2) - midPoint), ( UPPER_BOUND * np.sqrt(2)/2), FRONT_BOUND])

        arcLwBack = dict(boundPoints = [3,13], arcPoint = [-(LOWER_BOUND *(np.sqrt(2)/2) - midPoint), (LOWER_BOUND * np.sqrt(2)/2), BACK_BOUND])
        arcLwFront = dict(boundPoints = [9,16], arcPoint = [-(LOWER_BOUND * (np.sqrt(2)/2) - midPoint), (LOWER_BOUND * np.sqrt(2)/2), FRONT_BOUND])

        self.__Arcs.append(arcUpBack)
        self.__Arcs.append(arcUpFront)

        self.__Arcs.append(arcLwBack)
        self.__Arcs.append(arcLwFront)

    """" ONLY AFTER BLOCKS """           
    def CalculateBoundaries(self):
        
        if(self.__blocksAmount == 0):
            print("BLOCKS ERROR")
            return 

        def GetFaceFromBlock(block, face: Face):
            
            match face:
                case Face.LEFT:
                    return [block[4], block[0], block[3], block[7]]
                case Face.RIGHT:
                    return [block[5], block[1], block[2], block[6]]
                case Face.UP:
                    return [block[7], block[3], block[2], block[6]]
                case Face.DOWN:
                    return [block[4], block[0], block[1], block[5]]
                case Face.BACK:
                    return [block[0], block[1], block[2], block[3]]
                case Face.FRONT:
                    return [block[4], block[5], block[6], block[7]]
                

            return [-1, -1, -1 , -1]
        
        
        FirstUpperBlock = self.__Blocks[0]
        SecondUpperBlock = self.__Blocks[1]

        FirstLowerBlock = self.__Blocks[2]
        SecondLowerBlock = self.__Blocks[3]

        inlet = [
            GetFaceFromBlock(FirstUpperBlock, Face.UP),
            GetFaceFromBlock(FirstLowerBlock, Face.DOWN),
            GetFaceFromBlock(SecondUpperBlock, Face.UP),
            GetFaceFromBlock(SecondLowerBlock, Face.DOWN)
        ]

        outlet = [
            GetFaceFromBlock(SecondUpperBlock, Face.RIGHT),
            GetFaceFromBlock(SecondLowerBlock, Face.RIGHT)
        ]

        fixedWalls = [
                GetFaceFromBlock(FirstUpperBlock, Face.DOWN),
                GetFaceFromBlock(SecondUpperBlock, Face.DOWN),
                GetFaceFromBlock(FirstLowerBlock, Face.UP),
                GetFaceFromBlock(SecondLowerBlock, Face.UP),
                ]
        

        fixedWallsDict = dict(name = "walls", type = "wall", faces = fixedWalls)
        inletDict = dict(name = "inlet", type = "patch", faces = inlet)
        outletDict = dict(name = "outlet", type = "patch", faces = outlet)
        self.__Boundaries.append(fixedWallsDict)
        self.__Boundaries.append(inletDict)
        self.__Boundaries.append(outletDict)
        
    def GetSpline(self, splineNumber: int):
        return self.__SplinePoints[splineNumber], self.__Splines[splineNumber]
    
    def GetBlock(self, blockNumber: int):
        return self.__Blocks[blockNumber]
    
    def GetVertex(self, vertexNumber: int):
        return self.__Vertices[vertexNumber]

    def GetBoundary(self, boundaryNumber: int):
        return self.__Boundaries[boundaryNumber]
        
    def GetArc(self, arcNumber: int):
        return self.__Arcs[arcNumber]

    def GetVerticesAmount(self):
        return self.__VerticesAmount
    
    def GetBlocksAmount(self):
        return self.__BlocksAmount
    
    def GetSplinesAmount(self):
        return self.__SplinesAmount
    
    def GetBoundariesAmount(self):
        return len(self.__Boundaries)
    
    def GetArcsAmount(self):
        return len(self.__Arcs)
    
    __Vertices = []
    __Splines = []
    __SplinePoints = []
    __Arcs = []
    __Blocks = []
    __Boundaries = []

    __xu = []
    __yu = []
    __xl = []
    __yl = []

    __VerticesAmount = 0
    __SplinesAmount = 0
    __BlocksAmount = 0
  
class BlockMeshGenerator:
    def __init__(self, airfoilmesh: AirfoilMesh):

        self.__AirfoilMesh = airfoilmesh

        self.WriteHeader()
        self.WriteVertices()
        self.WriteBlocks()
        self.WriteSplines()
        self.WriteBoundaries()
        
    def WriteHeader(self):
        with open("blockMeshHeader.txt", "r") as file:
            header = file.read()

        with open("blockMeshDict.txt", "w") as file:
            file.write(header)
    
    def WriteVertices(self):
        file = open('blockMeshDict.txt', 'a')
        verticesAmount = self.__AirfoilMesh.GetVerticesAmount()
        
        file.write("\n")
        file.write("vertices\n(\n")
        for i in range(0, verticesAmount, 1):
            vertex = self.VertexToString(self.__AirfoilMesh.GetVertex(i), i)
            file.write(vertex)
        
        file.write(");\n") 
        file.close()

    def WriteBlocks(self):
        file = open('blockMeshDict.txt', 'a')
        blocksAmount = self.__AirfoilMesh.GetBlocksAmount()
        
        file.write("\n")
        file.write("blocks\n(\n")
        for i in range(0, blocksAmount, 1):
            block = self.BlockToString(self.__AirfoilMesh.GetBlock(i))
            file.write(block)
            file.write("\n")
        
        file.write(");\n")
            
        file.close()

    def WriteBoundaries(self):
        boundariesAmount = self.__AirfoilMesh.GetBoundariesAmount()

        file = open('blockMeshDict.txt', 'a')
        file.write("\n")
        file.write("boundary\n(\n")
        for i in range(0, boundariesAmount, 1):
            boundary = self.__AirfoilMesh.GetBoundary(i)
            file.write(self.BoundaryToString(boundary))

        file.write("\n);\n")

        file.write("defaultPatch\n{\n\tname frontAndBack;\n\ttype empty;\n}\n")													
        file.close()

    def WriteSplines(self):
        splinesAmount = self.__AirfoilMesh.GetSplinesAmount()
        arcAmount = self.__AirfoilMesh.GetArcsAmount()

        file = open('blockMeshDict.txt', 'a')
        file.write("\n")
        file.write("edges\n(\n")

        for i in range(0, arcAmount, 1):
            arcStr = self.ArcToString(i)
            file.write(arcStr)

        for i in range(0, splinesAmount, 1):
            splineStr = self.SplinesToString(i)
            file.write(splineStr)
            
        file.write("\n);\n")
        file.close()

    def VertexToString(self, vertex, vertexNumber):
        vertexString = "\t( " + str(vertex[0]) + " " + str(vertex[1]) + " " + str(vertex[2]) + " ) // " + str(vertexNumber) + "\n"
        return vertexString
    
    def BlockToString(self, block, grid = (20, 20, 1), grading = (1, 1, 1)):
        blockString = "\thex ( "
        for i in range(0, 8, 1):
            blockString += str(block[i]) + " "
        
        blockString += " ) ( " + str(grid[0]) + " " + str(grid[1]) + " " + str(grid[2]) + " ) simpleGrading "
        blockString += "( " + str(grading[0]) + " " + str(grading[1]) + " " + str(grading[2]) +  " )"
        return blockString
    
    def SplinesToString(self, splineNumber):
        splinePair, splinePoints = self.__AirfoilMesh.GetSpline(splineNumber)
        splineString = "\n\tpolyLine " + str(splinePair[0]) + " " +  str(splinePair[1]) + " (\n"
        
        for i in range(0, len(splinePoints), 1):
            splineString += "\t\t( " + str(splinePoints[i][0]) + " " + str(splinePoints[i][1]) + " " + str(splinePoints[i][2])  + " )\n"

        splineString += "\t)"
        return splineString
    
    def BoundaryToString(self, boundary):


        boundaryString = "\t" + str(boundary["name"])
        boundaryString += "\n\t{\n"
        boundaryString += "\t\ttype " + str(boundary["type"]) + ";\n"
        boundaryString += "\t\tfaces\n"
        boundaryString += "\t\t(\n"

        faces = boundary["faces"]
        
        for i in range(0, len(faces), 1):
            boundaryString += "\t\t\t(" + " " + str(faces[i][0]) + " " + str(faces[i][1]) + " " + str(faces[i][2]) + " " + str(faces[i][3]) + " )\n"

        boundaryString += "\t\t);\n"
        boundaryString += "\t}\n"

        return boundaryString

    def ArcToString(self, arcNumber: int):
        arc = self.__AirfoilMesh.GetArc(arcNumber)
        arcString = "\tarc " + str(arc["boundPoints"][0]) + " " + str(arc["boundPoints"][1]) + " "
        arcString += "( " + " " + str(arc["arcPoint"][0]) + " " + str(arc["arcPoint"][1]) + " " + str(arc["arcPoint"][2]) + " )\n"
        return arcString

S = Airfoil("2412", 200)
D = AirfoilMesh(S, 4)
G = BlockMeshGenerator(D)



__ NACA AIRFOIL 2412, MaxCamber(%) 0.02, MaxCamberPosition(%) 0.4 Thickness(%) :0.12 __ Points 200
[13, 14, 2, 12, 16, 17, 8, 15]
